<a href="https://colab.research.google.com/github/tomonari-masada/course2025-nlp/blob/main/08_text_classification_with_LLM_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLMを使ったテキスト分類

* LLMをfreezeさせたままの状態で、分類用のヘッドだけを訓練する。
* 事前学習済みのautoregressiveな言語モデルを使う。
  * 最終レイヤの出力を平均したものをテキストの埋め込みとして使う。

## 準備

In [ ]:
import time
import torch
from torch import nn
from torch.utils.data import DataLoader
from datasets import load_dataset, DatasetDict
from transformers import set_seed, AutoModel, AutoTokenizer

set_seed(0)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

## データセット
* AG Newsデータセットを使う。

In [ ]:
ds = load_dataset("ag_news")
train_valid = ds["train"].train_test_split(test_size=0.05)
ds = DatasetDict({
    "train": train_valid["train"],
    "valid": train_valid["test"],
    "test": ds["test"],
})

ds

In [ ]:
unique_labels = set([label for label in ds["train"]["label"]])
print(unique_labels)
num_class = len(unique_labels)

ag_news_label = { 0: "World", 1: "Sports", 2: "Business", 3: "Sci/Tec" }

## LLM
* `LiquidAI/LFM2-350M`を使う。
  * 軽量なautoregressive LLM。

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("LiquidAI/LFM2-350M")
model = AutoModel.from_pretrained("LiquidAI/LFM2-350M").to(device)

for param in model.parameters():
  param.requires_grad = False

In [ ]:
params = 0
for p in model.parameters():
  params += p.numel()

print(f"The model has {params:,} parameters")

In [ ]:
model

### モデルの詳細を知る方法＝ソースを読む
* guthubにログインしてtransformersライブラリで`Lfm2ForCausalLM`を検索
  * 「/」キーを押してから検索語を入力すればよい。
* あるいは、下の場所でモデルがありそうなディレクトリを探す。
  * https://github.com/huggingface/transformers/tree/main/src/transformers/models

## トークナイザ

In [ ]:
tokenizer(ds["train"]["text"][:3])

In [ ]:
tokenizer(ds["train"]["text"][:3], padding=True)

## DataLoader

In [ ]:
def collate_fn(batch):
  texts = []
  labels = []
  for sample in batch:
    texts.append(sample["text"])
    labels.append(sample["label"])
  tokenized = tokenizer(texts, padding=True, return_tensors="pt")
  return tokenized.to(device), torch.tensor(labels).to(device)

In [ ]:
BATCH_SIZE = 50

train_dataloader = DataLoader(ds["train"], batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
valid_dataloader = DataLoader(ds["valid"], batch_size=BATCH_SIZE, collate_fn=collate_fn)
test_dataloader = DataLoader(ds["test"], batch_size=BATCH_SIZE, collate_fn=collate_fn)

In [ ]:
next(iter(train_dataloader))

In [ ]:
tokenized, labels = next(iter(train_dataloader))
output = model(input_ids=tokenized["input_ids"], attention_mask=tokenized["attention_mask"])

In [ ]:
output

In [ ]:
output.last_hidden_state.shape

In [ ]:
class TextClassificationModel(nn.Module):
  def __init__(self, model, num_class):
    super(TextClassificationModel, self).__init__()
    self.model = model
    self.fc = nn.Linear(model.config.hidden_size, num_class).to(model.device)

  # forward pass
  def forward(self, input_ids, attention_mask):
    output = self.model(input_ids=input_ids, attention_mask=attention_mask)
    embedded = (output.last_hidden_state * attention_mask.unsqueeze(-1)).sum(1)
    embedded = embedded / attention_mask.sum(1, keepdim=True)
    return self.fc(embedded)

In [ ]:
clf = TextClassificationModel(model, num_class)

In [ ]:
def train(dataloader, optimizer, criterion):
  clf.train()
  total_acc, total_count = 0, 0
  log_interval = 10 # ログ情報を表示する間隔
  start_time = time.time()

  for idx, (tokenized, labels) in enumerate(dataloader):
    optimizer.zero_grad()
    logits = clf(tokenized["input_ids"], tokenized["attention_mask"])
    loss = criterion(logits, labels)
    loss.backward()
    optimizer.step()
    total_acc += (logits.argmax(1) == labels).sum().item()
    total_count += labels.size(0)
    if idx % log_interval == 0 and idx > 0:
      elapsed = time.time() - start_time
      print(
          f"||| {idx:5d}/{len(dataloader):5d} batches | "
          f"time: {elapsed:5.2f}s | "
          f"accuracy {total_acc / total_count:8.3f}"
      )
      total_acc, total_count = 0, 0
      start_time = time.time()

In [ ]:
def evaluate(dataloader, criterion):
  clf.eval()
  total_acc, total_count = 0, 0

  with torch.no_grad():
    for idx, (tokenized, labels) in enumerate(dataloader):
      logits = clf(tokenized["input_ids"], tokenized["attention_mask"])
      loss = criterion(logits, labels)
      total_acc += (logits.argmax(1) == labels).sum().item()
      total_count += labels.size(0)
  return total_acc / total_count

In [ ]:
epochs = 10
learning_rate = 1e-4

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(clf.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(epochs):
  epoch_start_time = time.time()
  train(train_dataloader, optimizer, criterion)
  accu_val = evaluate(valid_dataloader, criterion)
  print("-" * 59)
  elapsed = time.time() - epoch_start_time
  print(
      f"| end of epoch {epoch+1:3d} | "
      f"time: {elapsed:5.2f}s | "
      f"lr = {optimizer.param_groups[0]['lr']:.3f} | "
      f"validation accuracy {accu_val:8.3f}"
  )
  print("-" * 82)

In [ ]:
print("Checking the results of test dataset...")
accu_test = evaluate(test_dataloader, criterion)
print(f"test accuracy {accu_test:8.3f}")